In [1]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Wrangling
import pandas as pd
import numpy as np
import wrangle
import acquire
# Statistical Tests
import scipy.stats as stats

# Visualizing
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from sklearn.model_selection import learning_curve


pd.options.display.float_format = '{:20,.2f}'.format
pd.set_option('display.max_columns', 80)

In [2]:
df= wrangle.wrangle_zillow()

In [3]:
df.head()

,parcelid,bathrooms,bedrooms,buildingquality,square_feet,fips,latitude,longitude,lot_size,regionidcity,regionidcounty,regionidzip,roomcnt,unitcnt,age,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,logerror,transactiondate,heatingorsystemdesc,county
0,10711855,2.00,3.00,8.00,"2,107.00",6037,"34,222,559.00","-118,617,387.00","9,158.00","12,447.00","3,101.00","96,339.00",0.00,1.00,45.00,"249,655.00","624,139.00","2,016.00","374,484.00","7,659.36",-0.01,2017-07-07,Central,Los_Angeles
1,10711877,2.00,4.00,8.00,"1,882.00",6037,"34,220,261.00","-118,616,409.00","9,035.00","12,447.00","3,101.00","96,339.00",0.00,1.00,45.00,"253,000.00","660,000.00","2,016.00","407,000.00","8,123.91",0.02,2017-08-29,Central,Los_Angeles
2,10711888,2.00,4.00,8.00,"1,882.00",6037,"34,222,491.00","-118,616,854.00","9,800.00","12,447.00","3,101.00","96,339.00",0.00,1.00,45.00,"257,591.00","542,923.00","2,016.00","285,332.00","6,673.24",0.08,2017-04-04,Central,Los_Angeles
3,10711910,2.00,3.00,8.00,"1,477.00",6037,"34,221,864.00","-118,615,739.00","11,285.00","12,447.00","3,101.00","96,339.00",0.00,1.00,57.00,"57,968.00","78,031.00","2,016.00","20,063.00","1,116.46",-0.04,2017-03-17,Central,Los_Angeles
4,10711923,2.00,4.00,8.00,"1,918.00",6037,"34,220,619.00","-118,615,253.00","11,239.00","12,447.00","3,101.00","96,339.00",0.00,1.00,57.00,"167,869.00","415,459.00","2,016.00","247,590.00","5,239.85",-0.01,2017-03-24,Central,Los_Angeles


In [4]:
X_train, y_train, X_validate, y_validate, X_test, y_test = wrangle.data_split(df,stratify_by='logerror')

First Hypothesis :  <br>
𝐻0: There is no difference in log error based on a properties square footage. <br>
𝐻𝑎: Properties with a square footage less than 2800 square footage have a lower log error than larger properties. 
<br><br>
Second Hypothesis: <br>
𝐻0 : There is no difference in median log error in properties built in different years. <br>
𝐻𝑎 : Properties that were built after 1975 have a higher median log error than properties built before 1975.
<br><br>
Third Hypothesis: <br>
𝐻0 : There is no correlation between latitude and log error. <br>
𝐻𝑎 : Properties that are more southern have a higher log error.
